In [4]:
from __future__ import division
import numpy as np
import h5py
import pandas as pd
from readtree import DHaloReader as DHalo
from readtree import read_tree_keys
from SubHalos import SubHalos as SHfuncs
from SubHalos import read_subhalo_keys
from SubHalos import read_group_keys

In [5]:
## SubFind
#sh_orig_file = '/cosma6/data/dp004/dc-arno1/SZ_project/'\
#               'dark_matter_only/L62_N512_GR/'
#sh_reor_file = '/cosma5/data/dp004/dc-beck3/Galaxy_Evolution/'\
#               'SubFind/dm_only/L62_N512_GR/subfind.0.hdf5'
#mt_file = '/cosma5/data/dp004/dc-oles1/dhalo/out/trees/GR/'\
#          'treedir_075/tree_075.0.hdf5'  # 71 is 41, simply because

sh_orig_file = "/gpfs/data/Eagle/yanTestRuns/MergerTree/" \
            "Dec14/L0100N1504/EAGLE_L0100N1504_db.hdf5"
mt_file = "/gpfs/data/Eagle/yanTestRuns/MergerTree/" \
            "Dec14/L0100N1504/EAGLE_L0100N1504_db.hdf5"


In [6]:
filename = mt_file
columns = [
    "nodeIndex",
    "snapshotNumber",
    "hostIndex",
    "descendantHost",
    "descendantIndex",
    "isMainProgenitor",
    "mainProgenitorIndex",
    "isInterpolated"
]        

with h5py.File(filename, "r") as data_file:

    # Find dimensionality of keys
    columns_1dim = [] 
    columns_2dim = [] 
    for column in columns:
        if len(data_file["/haloTrees/%s" % column].shape) == 1:
            columns_1dim.append(column)
        else:
            columns_2dim.append(column)

    # 1D keys
    data = pd.DataFrame(
        {
            column: data_file["/haloTrees/%s" % column].value
            for column in columns_1dim
        },
        columns=columns_1dim
    ).set_index("nodeIndex")
    del columns_1dim

    # 2D keys
    for column in columns_2dim:
        if column == 'position':
            pos = data_file["/haloTrees/%s" % column].value
            data['X'] = pd.Series(pos[:, 0], index=data.index)
            data['Y'] = pd.Series(pos[:, 1], index=data.index)
            data['Z'] = pd.Series(pos[:, 2], index=data.index)
    del columns_2dim

    ## eliminate fake elements with isIntegrated=1
    #data = data[data.isInterpolated != 1]

KeyError: 'Unable to open object (component not found)'

In [8]:
filename = mt_file
columns = [
    "DhaloIndex",
    "GroupIndex",
    "SnapNum",
    "DescendantID",
    "HaloID",
    "LastProgID",
    "TopLeafID"
]

data_file = h5py.File(filename, 'r')
column_mt = []
column_sh = []
for column in columns:
    if column in data_file['MergerTree']:
        column_mt.append(column)
    else:
        column_sh.append(column)

data = pd.DataFrame(
    {
        column: data_file["/MergerTree/%s" % column].value
        for column in column_mt
    },
    columns=column_mt
).set_index("HaloID")
#.set_index(data_file["/Merger/HaloID"].value)

for column in column_sh:
    data[column] = pd.Series(data_file["/Subhalo/%s" % column].value,
                             index=data.index)
data = data.rename(index=str,
                   columns={"SnapNum": "snapnum", #"HaloID": "nodeIndex",
                            "DescendantID" : "descendantIndex"})

In [9]:
print('numpy.version.version', np.version.version)
print('pandas.version.version', pd.__version__)
from itertools import chain

snapcount = 0
z2 = 26
z1 = 23        
for ss in range(z2, z1, -1):

    if ss == z2:
        _indx = np.where(data.snapshotNumber.values == ss)
        nodeID = data.index.values[_indx]
        _indx = np.where(data.snapshotNumber.values == ss-1)
        nodeID_prog = data.index.values[_indx]
        nodeID_prog_desc = data.descendantIndex.values[_indx]

        # Initiliaze Array
        progcounts = np.zeros((len(nodeID), z2-z1+1))
        nodeID_obs_ref = [None]*len(nodeID)

        # nodeID_prog_desc_unic is sorted
        nodeID_prog_desc_unic, count = np.unique(nodeID_prog_desc,
                                                 return_counts=True)
        # remove -1's
        nodeID_prog_desc_unic=nodeID_prog_desc_unic[1:]; count=count[1:]

        # Nr. of progenitors for sub-&halos at snapshot z2
        _indx_now = np.arange(nodeID.shape[0])[np.in1d(
            nodeID, nodeID_prog_desc_unic,
            assume_unique=True)]
        now_sort_indx = np.argsort(nodeID[_indx_now])
        pro_sort_indx = np.argsort(nodeID_prog_desc_unic)
        progcounts[_indx_now[now_sort_indx], snapcount] = count[pro_sort_indx]
            
        # sort nodeID_prog to nodeID
        df_prog = pd.DataFrame({'nodeID' : nodeID_prog,
                                'nodeID_target' : nodeID_prog_desc})
        print('nodeID_prog_desc', len(np.unique(nodeID_prog_desc))/len(nodeID_prog_desc))

        nodeID_target = nodeID_prog_desc
        df_target = pd.DataFrame({'nodeID':nodeID})
    else:
        df_now = df_prog
        _indx = np.where(data.snapshotNumber.values == ss-1)
        df_prog = pd.DataFrame({'nodeID' : data.index.values[_indx]})
        nodeID_prog_desc = data.descendantIndex.values[_indx]
 
        progcounts_local = np.zeros(df_now['nodeID'].size)
        nodeID_prog_desc_unic, count = np.unique(nodeID_prog_desc,
                                                 return_counts=True)
        # remove -1's
        nodeID_prog_desc_unic=nodeID_prog_desc_unic[1:]; count=count[1:]
        
        s = pd.Index(df_now['nodeID'].tolist())
        _indx_now = s.get_indexer(list(nodeID_prog_desc_unic))
        now_sort_indx = np.argsort(df_now['nodeID'].values[_indx_now])
        pro_sort_indx = np.argsort(nodeID_prog_desc_unic)
        progcounts_local[_indx_now[now_sort_indx]] = count[pro_sort_indx]
        df_now['progcount'] = pd.Series(progcounts_local,
                                        index=df_now.index, dtype=int)

        # Nr. of progenitors for sub-&halos at snapshot z2
        df_inter = df_now.groupby(['nodeID_target'], as_index=False)['progcount'].sum()
        df_inter = df_inter[df_inter['nodeID_target'] >= 1e11]  # only real progeniteurs
        df_inter = df_inter.drop_duplicates(subset=['nodeID_target'], keep='first')
        
        s = pd.Index(df_target['nodeID'].tolist())
        _indx_now = s.get_indexer(df_inter['nodeID_target'].tolist())
        now_sort_indx = np.argsort(df_target['nodeID'].values[_indx_now])
        pro_sort_indx = np.argsort(df_inter['nodeID_target'].values)
        progcounts[_indx_now[now_sort_indx], snapcount] = df_inter['progcount'].values[pro_sort_indx]

        # sort nodeID_prog to nodeID
        #s = pd.Index(df_now['nodeID'].tolist())
        #_indx_now = s.get_indexer(list(nodeID_prog_desc_unic))
        #df_now['nodeID_target'].values[_indx_now]
        
        obs_ref_local = np.zeros(df_prog['nodeID'].size)
        for ii in range(len(nodeID_prog_desc_unic)):
            tarID = df_now.loc[df_now['nodeID'] == nodeID_prog_desc_unic[ii], 'nodeID_target']
            _indx = np.where(nodeID_prog_desc == nodeID_prog_desc_unic[ii])
            obs_ref_local[_indx] = int(tarID)
        df_prog['nodeID_target'] = pd.Series(obs_ref_local,
                                             index=df_prog.index)

    print('progcounts', np.mean(progcounts[:, snapcount]), snapcount)
    snapcount += 1
del nodeID_prog, nodeID_prog_desc
del df_target, df_now, df_inter, df_prog
print('progcounts', progcounts)

numpy.version.version 1.14.5
pandas.version.version 0.21.0
redshift: 71
nodeID_prog_desc 0.9775124947873552
progcounts 0.9976456886673721 0
redshift: 70
get_indexer [  8003   8877   8004 ... 148417   1965  58412]
progcounts 0.9970320649689745 1
redshift: 69
get_indexer [  8325   8326   8327 ... 158770 158775   4096]
progcounts 0.9775088130138316 2
redshift: 68
get_indexer [  8608   8609   8611 ... 162314   1559   2044]
progcounts 0.9813971835924536 3
progcounts [[0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 0.]]


In [ ]:
        #_indx_now = np.arange(df_target['nodeID'].size)[np.isin(df_target['nodeID'].values,
        #                                                        df_inter['nodeID_target'].values,
        #                                                        assume_unique=False)]
        #_indx_prog = np.arange(df_inter['nodeID_target'].size)[np.isin(df_inter['nodeID_target'].values,
        #                                                               df_target['nodeID'].values,
        #                                                               assume_unique=False)]
        #_indx_now = dznter['nodeID_target'].tolist())].index        
        #_indx_now = []
        #for ii in range(df_inter['nodeID_target'].size):
        #    _indx = np.where(df_target['nodeID'].values == df_inter['nodeID_target'].values[ii])
        #    _indx_now.append(_indx[0])
        #_indx_now = np.unique(np.fromiter(chain.from_iterable(_indx_now), dtype='int'))